# Analyzing Relational Database (SQL)

## 1. Remove duplicate records And Identify Primary Keys

In [ ]:
import pymysql
pymysql.install_as_MySQLdb()
%reload_ext sql
%local_host_address
%sql USE data_base_name;

In [ ]:
# To check if a table has duplicate records, I used the following code. If the two counts return the same number, we can confirm that all the records are unique. However, if
## the two counts return different numbers, we know that there are duplicates.
%%sql
SELECT COUNT(*)
FROM table_1

SELECT COUNT(*)
FROM (SELECT DISTINCT * FROM table_1) AS SubQ

In [ ]:
#To figure out which column can help uniquely identify records, I researched the documentation of the database and tried different combination of columns using the
## following code. If the COUNT(*) within each group is equal to one, we know that the columns selected uniquely identify each row.
%%sql
WITH CTE1 AS (SELECT DISTINCT * FROM table_1)
    SELECT column_1, column_2, COUNT(*)
    FROM CTE1
    GROUP BY column_1, column_2
    ORDER BY COUNT(*) DESC
    LIMIT 10;

In [ ]:
# Based on the code in previous chunk, I identified those columns that, when combined, can uniquely identify each row. They are also known as the primary keys.
## Partitioned by these columns, I assigned each row a number and removed the duplicates based on the number.
%%sql
WITH CTE1 AS (SELECT *, ROW_NUMBER() OVER(PARTITION BY column_1, column_2) AS RowNum FROM table_1)
    SELECT *
    FROM CTE1
    WHERE RowNum = 1;

## 2. Searching for Potential Issues in the Database

In [ ]:
# To check if there is any NULL value within a column
%%sql
WITH CTE1 AS
    (SELECT DISTINCT * FROM table_1)
SELECT column_1
FROM CTE1
WHERE column_1 IS NULL;

# To group the values within a column and check the count of each group. This helps to identify if tthe count distribution is abnormal
WITH CTE1 AS
    (SELECT DISTINCT * FROM table_1)
SELECT column_1, COUNT(*)
FROM CTE1
GROUP BY column_1
ORDER BY COUNT(*) DESC;

# To check the maximum and minimum value of a column. When there are too many distinct category within a column, using this method is more efficient
SELECT MAX(column_1), MIN(column_1)
FROM table_1;

# If there is two datetime type column, one denoted in minutes and another denoted in days, we can convert one of them to see if the two are equal to each other
SELECT *
FROM table_1
WHERE ((period_days > period_minutes / 60 / 24 + 1)
    OR (period_days < period_minutes / 60 / 24 - 1));

# If some values within column are mistyped and have extra sign, say "@" , at the beginning, we remove these sign to avoid double-counting
SELECT *, CASE WHEN SUBSTR(column_1, 1, 1) = '@' column_1 = SUBSTR(column_1, 2) ELSE column_1 END AS corrected_column_1
FROM table_1;

## 3. Summarize the data

In [ ]:
# Remove duplicates in table_1 and store the values in CTE2, then summarize the data in CTE2 to show the structure of customer base and since when have they stayed
## with the company
%%sql
WITH
CTE1 AS
    (SELECT DISTINCT *
     FROM table_1),
CTE2 AS
    (SELECT CTE1.*
     FROM CTE1 LEFT JOIN
    (SELECT CTE1.column_1, COUNT(*)
          FROM table_1
          GROUP BY CTE1. column_1
          HAVING COUNT(*) > 1) 
    AS SubQ
     ON CTE1.column_1 = SubQ.column_1
     WHERE SubQ.column_1 IS NULL
         OR
     (SubQ.column_1 IS NOT NULL AND SubQ.column_2 != 'value_1')
SELECT
     YEAR(column_3) AS Year,
     MONTH(column_3) AS Month,
     COUNT(*) AS Rec_Num,
     SUM(CASE WHEN column_4 = 'value_2' THEN 1 ELSE 0 END) / COUNT(*) AS Percentage
FROM
    CTE2
WHERE
     column_5 != 'value_3'
GROUP BY
     Year,
     Month
ORDER BY
     Year,
     Month;

In [ ]:
# Check the discrepancy between one column from a table and one calculated column based on information from another table to get a better understanding of the dataset
%%sql
WITH CTE1 AS (SELECT DISTINCT * FROM table_1),
            CTE2 AS (SELECT column_1_a, COUNT(*) FROM CTE1 GROUP BY column_1_a)
SELECT CTE2.column_1_a, table_2.column_1_b, CTE2.COUNT(*), table_2.column_2
FROM table_2 LEFT JOIN CTE2 ON table_2.column_1_b = CTE2.column_1_a
WHERE CTE2.COUNT(*) != CAST(table_2.column_2 AS INT);

In [ ]:
# Combine three tables and summarize the data to illustrate company's performance over each month within each year
%%sql
WITH
CTE1 AS (SELECT DISTINCT *
                 FROM table)1,
CTE2 AS (SELECT CTE1.*
                 FROM CTE1
                      LEFT JOIN (SELECT column_1, COUNT(*) FROM CTE1 GROUP BY column_1 HAVING COUNT(*) > 1) AS SubQ1
                     ON CTE1.column_1 = SubQ1.column_1
                 WHERE SubQ1.column_1 IS NULL
                 OR SubQ1.column_1 IS NOT NULL
                 AND CTE1.column_2 != 'N/A'),
CTE3 AS (SELECT DISTINCT *
                  FROM table_2),
CTE4 AS (SELECT column_3, COUNT(*) AS count_1
                 FROM CTE3
                 GROUP BY column_3),
CTE5 AS (SELECT table_3.column_3, table_3.column_1, COALESCE(CTE4.count_1, 0) AS count_2
                 FROM table_3 LEFT JOIN CTE4 ON table_3.column_3 = CTE4.column_3),
CTE6 AS (SELECT column_1, SUM(count_2) AS count_3
                 FROM CTE5
                 GROUP BY column_1),
CTE7 AS (SELECT CTE2.column_1, CTE2.column_4, CTE2.column_5, COALESCE(CTE6.count_3, 0) AS count_final 
                 FROM CTE2 LEFT JOIN CTE6 ON CTE2.column_1 = CTE6.column_1
                 WHERE CTE2.column_6 = 0 OR CTE2.column_6 IS NULL)
SELECT
    YEAR(column_4) AS Year,
    MONTH(column_5) AS Month,
    COUNT(*),
    AVG(count_final),
    SUM(CASE WHEN column_5 = value_1 THEN 1 ELSE 0 END) / COUNT(*) AS PCT_1,
    SUM(CASE WHEN column_5 = value_2 THEN 1 ELSE 0 END) / COUNT(*) AS PCT_2, 
    SUM(CASE WHEN column_5 = value_3 THEN 1 ELSE 0 END) / COUNT(*) AS PCT_3,
    SUM(CASE WHEN column_5 = value_4 THEN 1 ELSE 0 END) / COUNT(*) AS PCT_4,
    SUM(CASE WHEN column_5 = value_5 THEN 1 ELSE 0 END) / COUNT(*) AS PCT_5
FROM CTE7
GROUP BY Year, Month
ORDER BY Year, Month;

In [ ]:
# Check whether the services we provide are flawed by looking at the number of each type of service completed under different customer type
%%sql
WITH
CTE1 AS (SELECT DISTINCT *
                 FROM table_2),
CTE2 AS (SELECT CTE1.column_7, CTE1.column_8, table_3.*
                 FROM CTE1 LEFT JOIN table_3
                     ON CTE1.column_3 = table_3.column_3),
CTE3 AS (SELECT DISTINCT *
                 FROM table_1),
CTE4 AS (SELECT CTE3.*
                 FROM CTE3 LEFT JOIN (SELECT column_1, COUNT(*) FROM CTE3 GROUP BY column_1 HAVING COUNT(*) > 1) AS SubQ1
                     ON CTE3.column_1 = SubQ1.column_1
                 WHERE SubQ1.column_1 IS NULL
                     OR SubQ1.column_1 IS NOT NULL
                     AND column_2 != 'N/A')
SELECT column_7, column_8, COUNT(*)
FROM CTE2 LEFT JOIN CTE4
    ON CTE2.column_1 = CTE4.column_1
WHERE CTE4.column_5 = 4
    AND CTE4.column_9 = 1
GROUP BY column_7, column_8;

In [ ]:
# Segment customers to verify if certain customer group characteristics lead to different service demand
%%sql
WITH 
CTE1 AS (SELECT DISTINCT *
                 FROM table_2),
CTE2 AS (SELECT table_3.*
                 FROM CTE1 LEFT JOIN table_3
                     ON CTE1.column_3 = table_3.column_3
                 WHERE table_3.column_6 = 0
                     OR table_3.column_6 IS NULL),
CTE3 AS (SELECT column_1, column_3, COUNT(*) AS count_1
                 FROM CTE2
                 GROUP BY column_1, column_3),
CTE4 AS (SELECT DISTINCT *
                 FROM table_1),
CTE5 AS (SELECT CTE4.*
                 FROM CTE4 LEFT JOIN
                     (SELECT column_1, COUNT(*)
                     FROM CTE4
                     GROUP BY column_1
                     HAVING COUNT(*) > 1) AS SubQ1
                 ON CTE4.column_1 = SubQ1.column_1
                 WHERE SubQ1.column_1 IS NULL
                     OR SubQ1.column_1 IS NOT NULL
                     AND column_2 != 'N/A'),
CTE6 AS (SELECT CTE5.column_1, COALESCE(CTE3.count_1, 0) AS count_final
                 FROM CTE5 LEFT JOIN CTE3 ON CTE5.column_1 = CTE3.column_1 
                WHERE CTE5.column_6 = 0
                     OR CTE5.column_6 IS NULL
                GROUP BY CTE3.column_1)
SELECT CASE 
    WHEN count_final = 0 THEN '1. 0 group'
    WHEN count_final < 4 AND total_test_final > 0 THEN '2. > 0 & < 4 group'
    WHEN count_final = 4 THEN '3. = 4 group'
    WHEN count_final > 4 AND total_test_final < 20 THEN '4. > 4 & < 20 group'
    END AS group_name,
AVG(count_final)
FROM CTE6
GROUP BY group_name;